# DataLoader in PyTorch

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My Drive/HSE_DL_2021/04_week

__DataLoader__ -- класс в PyTorch, который позволяет итеративно проходить по датасету, отвечает за оркестрацию всего процесса работы с датасетом.

In [ ]:
from IPython import display
display.Image('images/DataLoaderParams.png')

- __dataset__ -- позволяет создать кастомные классы для работы с датасетом, где можно указать логику формирвоания батча.
- __sampler__ -- определяет порядок элементов из датасета, которые будут идти в батч, то есть это список индексов, объединенных в батч. Удобно переопределять, когда обучение распредленное.  
- __collate_fn__ -- позволяет сделать финальную предобработку над батчем данных. Если, например, в батч попали последовательности разных размеров, то после уже сбора батча, можно будет дополнить последовательности нулями относительно максимально длиной последовательности.



## Custom Dataset

In [ ]:
import pandas as pd
import pickle
import numpy as np

from torch.utils.data import DataLoader, Dataset, Sampler
from torch.utils.data.dataloader import default_collate

In [ ]:
BATCH_SIZE = 128
EPOCHS = 100

In [ ]:
with open('data/X_train_cat.pickle', 'rb') as f:
    X, target = pickle.load(f)

In [ ]:
class CustomDataset(Dataset):
    # Конструктор, где считаем датасет
    def __init__(self, dataset_path):
        with open(dataset_path, 'rb') as f:
            self.X, self.target = pickle.load(f)

        return
    
    # Переопределяем метод вычисление размера датасета
    def __len__(self):
        return len(self.X)

    # Переопределяем метод,
    # который достает по индексу наблюдение из датасет
    def __getitem__(self, idx):
        return self.X[idx], self.target[idx]
        

## Custom Sampler

In [ ]:
a = [1,2,3,4,5, 6, 7, 8]
print(np.random.permutation(a))
a

In [ ]:
class CustomSampler(Sampler):

    # Конструктор, где инициализируем индексы элементов
    def __init__(self, data):
        self.data_indices = np.random.permutation(len(data))

    def __len__(self):
        return len(self.data_indices)

    # Возращает итератор,
    # который будет возвращать индексы из перемешанного датасета
    def __iter__(self):
        return iter(self.data_indices)

## Custom Sampler

In [ ]:
def collate(batch):
    return default_collate(batch)

In [ ]:
def create_data_loader(train_dataset, train_sampler,
                       test_dataset, test_sampler):
    train_loader = DataLoader(dataset=train_dataset, sampler=train_sampler,
                              batch_size=BATCH_SIZE, collate_fn=collate,
                              shuffle=False)

    test_loader = DataLoader(dataset=test_dataset, sampler=test_sampler,
                             batch_size=BATCH_SIZE, collate_fn=collate,
                             shuffle=False)

    return train_loader, test_loader

In [ ]:
# Создаем объекты Custom Dataset и Sampler
train_ds = CustomDataset('data/X_train_cat.pickle')
train_sampler = CustomSampler(train_ds.X)

test_ds = CustomDataset('data/X_test_cat.pickle')
test_sampler = CustomSampler(test_ds.X)

In [ ]:
train_loader, test_loader = create_data_loader(train_ds, train_sampler,
                                               test_ds, test_sampler)

In [ ]:
def run_train():
    print('Run train')
    for epoch in range(EPOCHS):
        for features, labels in train_loader:
            print(features.shape, labels.shape)
            break

        # Run validation
        print('Run validation')
        for features, labels in test_loader:
            print(features.shape, labels.shape)
            break
            
        break

    return

In [ ]:
run_train()